<a href="https://colab.research.google.com/github/euns-tory/AIFFEL_quest_cr/blob/main/practice/jellyfish_hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p ~/.kaggle
!mv /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!mkdir -p /content/jellyfish

In [ ]:
!kaggle datasets download -d anshtanwar/jellyfish-types -p /content/drive/MyDrive/jellyfish

Dataset URL: https://www.kaggle.com/datasets/anshtanwar/jellyfish-types
License(s): Attribution 4.0 International (CC BY 4.0)


In [ ]:
!unzip /content/drive/MyDrive/jellyfish/jellyfish-types.zip -d /content/drive/MyDrive/jellyfish/

In [2]:
!pip install -q transformers datasets accelerate timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 데이터 전처리 및 학습 준비

In [3]:
from datasets import load_dataset, Dataset
from transformers import AutoImageProcessor, ViTForImageClassification, TrainingArguments, Trainer
from torchvision import transforms
from PIL import Image
import os

# 경로 설정
data_dir = "/content/drive/MyDrive/jellyfish"
class_names = sorted([
    name for name in os.listdir(data_dir)
    if os.path.isdir(os.path.join(data_dir, name)) and not name.startswith(".") and name != "Train_Test_Valid"
])
id2label = {i: name for i, name in enumerate(class_names)}
label2id = {v: k for k, v in id2label.items()}
print(id2label)

# 이미지 및 라벨 수집
image_paths, labels = [], []
for idx, cls in enumerate(class_names):
    cls_path = os.path.join(data_dir, cls)
    for fname in os.listdir(cls_path):
        if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            image_paths.append(os.path.join(cls_path, fname))
            labels.append(idx)

# Hugging Face Dataset 변환
dataset = Dataset.from_dict({"image": image_paths, "label": labels})
dataset = dataset.train_test_split(test_size=0.2)


{0: 'Moon_jellyfish', 1: 'barrel_jellyfish', 2: 'blue_jellyfish', 3: 'compass_jellyfish', 4: 'lions_mane_jellyfish', 5: 'mauve_stinger_jellyfish'}


## 전처리 및 모델 정의

In [4]:
processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std),
])

def transform_func(example):
    image = Image.open(example["image"]).convert("RGB")
    example["pixel_values"] = transform(image)
    return example

dataset = dataset.map(transform_func)

# 모델 불러오기 (PyTorch 기반)
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(class_names),
    id2label=id2label,
    label2id=label2id,
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Map:   0%|          | 0/720 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 학습

In [5]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-e7405cc4cd9f>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.141900,0.987313,0.894444
2,0.461300,0.486399,0.938889
3,0.249300,0.295063,0.966667
4,0.168300,0.227789,0.977778
5,0.153500,0.209185,0.988889


{'eval_loss': 0.2091846466064453,
 'eval_accuracy': 0.9888888888888889,
 'eval_runtime': 140.6363,
 'eval_samples_per_second': 1.28,
 'eval_steps_per_second': 0.085,
 'epoch': 5.0}

## 정확도 평가 및 grad-CAM

In [6]:
# 정확도 평가
eval_results = trainer.evaluate()
print(f"🔍 정확도: {eval_results['eval_accuracy']:.4f}")


🔍 정확도: 0.9889


In [7]:
!pip install -q grad-cam


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [9]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

# 이미지 로딩 함수
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    transformed = transform(image)
    return image, transformed.unsqueeze(0)  # (1, 3, 224, 224)

# Grad-CAM 생성 함수
def generate_gradcam(image_path, model, target_layer, label=None):
    model.eval()
    original_image, input_tensor = load_image(image_path)

    # Grad-CAM 설정
    cam = GradCAM(model=model, target_layers=[target_layer], use_cuda=torch.cuda.is_available())

    # 예측 결과를 기반으로 Grad-CAM 타겟 결정
    targets = None
    if label is not None:
        targets = [ClassifierOutputTarget(label)]

    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0]  # (224, 224)

    # 원본 이미지를 numpy로 변환
    rgb_img = np.array(original_image.resize((224, 224))) / 255.0

    # CAM 시각화
    cam_image = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

    # 시각화
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.title("Original")
    plt.imshow(original_image)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Grad-CAM")
    plt.imshow(cam_image)
    plt.axis('off')
    plt.show()
